# Assignment 3

## Group members
- Axel Gustafsson - gusgusaxa@student.gu.se
- Erdem Halil - gushaliler@student.gu.se

#### 1. Implement the K-means algorithm

#### 2. Apply the K-means algorithm to compress images

#### 3. Use AIC and BIC to choose K for Gaussian Mixture Models